In [2]:
import ccxt
import pprint

# with open("../binance.key") as f:
#     lines = f.readlines()
#     api_key = lines[0].strip()
#     api_secret = lines[1].strip()
   

api_key = "3o8od3V5ePVCnZ19u34KydThmSgbD2ekCV4yD9ZclRogzMvMl4RPFlfLW8fVpzFX"
api_secret = "ZRzqHcLKS7Lnemy5cwM2Cck0HRnguN59jQvzAqrrXAoZvGwS9AKZk1dHbvZqBXyA"

exchange = ccxt.binance(config={
    'apiKey': api_key,
    'secret': api_secret,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future'       # 선물 거래
    }
})
print(exchange)

markets = exchange.load_markets()
# pprint.pprint(markets)
# print(len(markets.keys()))
tickers = exchange.fetch_tickers()

# :usdt 종목 선별
symbols = tickers.keys()
usdt_symbols = [x for x in symbols if x.endswith("USDT")]

Binance


In [124]:
import pandas as pd

# usdt_symbols = ['BTC/USDT:USDT']
# symbol = 'BTC/USDT:USDT'
timeframe = '1d'

# pprint.pprint(ohlcv)

# 각 코인의 가격 데이터를 저장할 딕셔너리
prices_data = {}

# 각 심볼에 대해 OHLCV 데이터를 가져와서 딕셔너리에 저장
for symbol in usdt_symbols:
    ohlcv = exchange.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=2000)
    df = pd.DataFrame(ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')  # timestamp를 datetime으로 변환
    
    if len(df) > 365:
        print(symbol)
        prices_data[symbol] = df
# print(df)

BAKE/USDT:USDT
FOOTBALL/USDT:USDT
FTM/USDT:USDT
RVN/USDT:USDT
XEM/USDT:USDT
ROSE/USDT:USDT
IMX/USDT:USDT
1000XEC/USDT:USDT
GAL/USDT:USDT
SPELL/USDT:USDT
KNC/USDT:USDT
UNFI/USDT:USDT
DOT/USDT:USDT
1INCH/USDT:USDT
IOST/USDT:USDT
DGB/USDT:USDT
CTK/USDT:USDT
HBAR/USDT:USDT
KLAY/USDT:USDT
FLM/USDT:USDT
LUNA2/USDT:USDT
CRV/USDT:USDT
DENT/USDT:USDT
UNI/USDT:USDT
LPT/USDT:USDT
OP/USDT:USDT
ENJ/USDT:USDT
QTUM/USDT:USDT
IOTA/USDT:USDT
RUNE/USDT:USDT
ENS/USDT:USDT
ANKR/USDT:USDT
EGLD/USDT:USDT
BNB/USDT:USDT
BAT/USDT:USDT
XMR/USDT:USDT
DOGE/USDT:USDT
CHZ/USDT:USDT
VET/USDT:USDT
FIL/USDT:USDT
API3/USDT:USDT
BCH/USDT:USDT
MANA/USDT:USDT
MATIC/USDT:USDT
ALICE/USDT:USDT
SAND/USDT:USDT
COMP/USDT:USDT
TRX/USDT:USDT
STORJ/USDT:USDT
REN/USDT:USDT
TRB/USDT:USDT
ETC/USDT:USDT
LDO/USDT:USDT
APT/USDT:USDT
SXP/USDT:USDT
AUDIO/USDT:USDT
OCEAN/USDT:USDT
IOTX/USDT:USDT
ANT/USDT:USDT
ONT/USDT:USDT
NEO/USDT:USDT
XLM/USDT:USDT
JASMY/USDT:USDT
BAL/USDT:USDT
SOL/USDT:USDT
BAND/USDT:USDT
RLC/USDT:USDT
COTI/USDT:USDT
CT

In [5]:
sym_list = []
for key in prices_data:
    sym_list.append(key)

In [10]:
sym_list[0]

'BAKE/USDT:USDT'

In [109]:
i = 3
print(sym_list[i])
df = prices_data[sym_list[i]]
df['Future_Close'] = df['close'].shift(-7)  # 7일 후 종가
df['Target'] = (df['Future_Close'] > df['close']).astype(int)  # 현재보다 7일 후 가격이 높으면 1, 아니면 0
df.dropna(inplace=True)  # 결측치 제거

RVN/USDT:USDT


In [108]:
new_scaled_data

array([[0.74842875, 0.6960193 , 0.69591727, 0.66902857, 0.00729609],
       [0.66906639, 0.68138669, 0.64468886, 0.66428571, 0.00486022],
       [0.66443835, 0.80437405, 0.68760834, 0.81891429, 0.01442291],
       ...,
       [0.02896812, 0.02596108, 0.02474744, 0.024     , 0.01930709],
       [0.02411153, 0.02501705, 0.02642118, 0.02731429, 0.01182409],
       [0.0273683 , 0.02926522, 0.01434634, 0.02474286, 0.04379736]])

In [112]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import numpy as np

# 새 OHLCV 데이터 파일 불러오기
new_df = df

# 데이터 전처리
# 기존에 사용된 스케일러를 사용하여 동일한 방법으로 데이터 스케일링
scaler = MinMaxScaler(feature_range=(0, 1))
new_scaled_data = scaler.fit_transform(new_df[['open', 'high', 'low', 'close', 'volume']].values)

time_step = 30

timestamp = []
# 시퀀스 데이터 생성 함수
def create_dataset(X, time_step):
    Xs = []
    global timestamp
    for i in range(len(X) - time_step):
        Xs.append(X[i:(i + time_step)])
        # timestamp.append(new_df['datetime'][i])
    return np.array(Xs)

# 함수 호출
time_step = 30  # 모델 훈련에 사용된 time_step
X_new = create_dataset(new_scaled_data, time_step)

# 기존에 훈련된 모델 불러오기
model = load_model('gpt.h5')

# 새 데이터에서 모델 성능 평가
# 이 경우, 정확한 레이블이 없기 때문에 예측값만 생성
predictions = model.predict(X_new)

# 예측 결과 출력 또는 추가 분석
print(predictions)


28/28 [==============================] - 0s 3ms/step
[[8.17140520e-01]
 [8.60746622e-01]
 [8.85965347e-01]
 [8.98309767e-01]
 [9.05140042e-01]
 [9.08257186e-01]
 [9.07081068e-01]
 [9.00799930e-01]
 [8.86043072e-01]
 [8.63028467e-01]
 [8.37013364e-01]
 [7.98793137e-01]
 [7.30276644e-01]
 [6.19953036e-01]
 [4.96014804e-01]
 [3.85674685e-01]
 [3.14642459e-01]
 [2.73907632e-01]
 [2.59072423e-01]
 [2.67743379e-01]
 [3.08390528e-01]
 [3.83834422e-01]
 [4.92063940e-01]
 [5.71397185e-01]
 [6.01403534e-01]
 [5.97086012e-01]
 [6.05194271e-01]
 [6.06241763e-01]
 [6.08552992e-01]
 [6.14573181e-01]
 [6.25366807e-01]
 [6.33465230e-01]
 [6.37926161e-01]
 [6.41298354e-01]
 [6.42766595e-01]
 [6.47517323e-01]
 [6.53705001e-01]
 [6.60200000e-01]
 [6.66917682e-01]
 [6.72720551e-01]
 [6.79544985e-01]
 [6.84678197e-01]
 [6.88701749e-01]
 [6.90944552e-01]
 [6.92275524e-01]
 [6.93365633e-01]
 [6.94712877e-01]
 [6.98382318e-01]
 [7.02537537e-01]
 [7.08404839e-01]
 [7.15192795e-01]
 [7.23662317e-01]
 [7.3190385

In [123]:
predictions.astype(float).round(1)

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [0.8],
       [0.9],
       [0.9],
       [0.9],
       [0.9],
       [0.9],
       [0.9],
       [0.9],
       [0.9],
       [0.9],
       [0.8],
       [0.8],
       [0.7],
       [0.6],
       [0.5],
       [0.4],
       [0.3],
       [0.3],
       [0.3],
       [0.3],
       [0.3],
       [0.4],
       [0.5],
       [0.6],
       [0.6],
       [0.6],
       [0.6],
       [0.6],
       [0.6],
       [0.6],
       [0.6],
       [0.6],
       [0.6],
       [0.6],
       [0.6],
       [0.6],
       [0.7],
       [0.7],
       [0.7],
       [0.7],
       [0.7],
      

In [114]:
nan_array = np.full((time_step, 1), np.nan)
predictions = np.vstack((nan_array, predictions))
df['predict'] = pd.DataFrame(predictions)

In [115]:
df.dropna(inplace=True)  # 결측치 제거

In [116]:
# 데이터 생성
df['Target'] = df['Target'].replace(0, -1)
df['result'] = (df['Future_Close'] - df['close']) / df['close']
df['decision'] = df['predict'].round(0)
df['decision'] = df['decision'].astype(int)
df['success'] = df['Target'] * df['decision']
df['decision'] = df['decision'].replace(0, -1)

In [118]:
df

,datetime,open,high,low,close,volume,Future_Close,Target,predict,result,decision,success
60,2021-04-24,0.14465,0.14651,0.12975,0.13075,229760054.0,0.18446,1,0.625367,0.410784,1,1
61,2021-04-25,0.13076,0.14372,0.12118,0.12992,183824461.0,0.18867,1,0.633465,0.452201,1,1
62,2021-04-26,0.12995,0.16717,0.12836,0.15698,364157228.0,0.18364,1,0.637926,0.169831,1,1
63,2021-04-27,0.15706,0.17929,0.15358,0.17059,302571950.0,0.16089,-1,0.641298,-0.056861,1,-1
64,2021-04-28,0.17059,0.17782,0.15726,0.16410,163470632.0,0.17543,1,0.642767,0.069043,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
918,2023-08-30,0.01585,0.01590,0.01549,0.01555,262918585.0,0.01497,-1,0.746190,-0.037299,1,-1
919,2023-08-31,0.01555,0.01569,0.01476,0.01488,308849711.0,0.01522,1,0.749236,0.022849,1,1
920,2023-09-01,0.01487,0.01497,0.01446,0.01463,234357664.0,0.01504,1,0.754090,0.028025,1,1
921,2023-09-02,0.01464,0.01527,0.01457,0.01499,328465380.0,0.01510,1,0.756101,0.007338,1,1


In [119]:
(df['success'] == -1).sum()

447

In [120]:
(df['success'] == 1).sum()

275